In [1]:
# Install all required libraries
!pip install unbabel-comet
!pip install -q transformers accelerate evaluate bert_score rouge_score sacremoses nltk psutil pandas sacrebleu sentencepiece

from huggingface_hub import login

# Paste your token here
login(token='hugging face token goes here')

# Download NLTK data needed for BLEU and ROUGE
import nltk
nltk.download('punkt')

import time
import torch
import psutil
import logging
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration
import evaluate

logging.getLogger("transformers").setLevel(logging.ERROR)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


paragraph_data = [
    {"lang": "ar", "name": "Arabic", "src": "أنا أتقدم للحصول على تأشيرة سياحية لزيارة الولايات المتحدة. أنوي البقاء لمدة أسبوعين وزيارة معالم مشهورة مثل تمثال الحرية وساحة التايمز. سأعود إلى بلدي بعد انتهاء عطلتي.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "es", "name": "Spanish", "src": "Estoy solicitando una visa de turista para visitar los Estados Unidos. Tengo la intención de quedarme dos semanas y visitar lugares famosos como la Estatua de la Libertad y Times Square. Regresaré a mi país después de mis vacaciones.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "fr", "name": "French", "src": "Je demande un visa de touriste pour visiter les États-Unis. J’ai l’intention de rester deux semaines et de visiter des sites célèbres comme la Statue de la Liberté et Times Square. Je retournerai dans mon pays après mes vacances.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "ru", "name": "Russian", "src": "Я подаю заявку на туристическую визу для поездки в Соединенные Штаты. Я собираюсь остаться на две недели и посетить известные достопримечательности, такие как Статуя Свободы и Таймс-сквер. Я вернусь в свою страну после отпуска.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "ro", "name": "Romanian", "src": "Solicit o viză turistică pentru a vizita Statele Unite. Intenționez să stau două săptămâni și să vizitez repere populare precum Statuia Libertății și Times Square. Mă voi întoarce în țara mea după vacanță.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "bg", "name": "Bulgarian", "src": "Кандидатствам за туристическа виза за посещение на Съединените щати. Възнамерявам да остана две седмици и да посетя популярни забележителности като Статуята на свободата и Таймс Скуеър. Ще се върна в родината си след ваканцията.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "cs", "name": "Czech", "src": "Žádám o turistické vízum pro návštěvu Spojených států. Mám v úmyslu zůstat dva týdny a navštívit známá místa jako Sochu svobody a Times Square. Po dovolené se vrátím do své domovské země.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "da", "name": "Danish", "src": "Jeg ansøger om et turistvisum for at besøge USA. Jeg har til hensigt at blive i to uger og besøge kendte vartegn som Frihedsgudinden og Times Square. Jeg vender tilbage til mit hjemland efter ferien.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "de", "name": "German", "src": "Ich beantrage ein Touristenvisum, um die Vereinigten Staaten zu besuchen. Ich beabsichtige, zwei Wochen zu bleiben und bekannte Sehenswürdigkeiten wie die Freiheitsstatue und den Times Square zu besuchen. Nach meinem Urlaub kehre ich in mein Heimatland zurück.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "et", "name": "Estonian", "src": "Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
]

# Load the 418M model and tokenizer
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name).to(device)
model.eval()

# Load metrics
bleu_mod      = evaluate.load("bleu")
bertscore_mod = evaluate.load("bertscore")
meteor_mod    = evaluate.load("meteor")
rouge_mod     = evaluate.load("rouge")
ter_mod       = evaluate.load("ter")
comet_mod     = evaluate.load("comet")

# System monitoring
def measure_system():
    proc = psutil.Process()
    return proc.memory_info().rss / 1024**2, psutil.cpu_percent(interval=None)

# Translation function
def translate_m2m100(text: str, src_lang: str, tgt_lang: str = "en"):
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt").to(device)
    start = time.perf_counter()
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))
    mid = time.perf_counter()
    pred = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    end = time.perf_counter()
    return pred, mid - start, end - start, len(tokenizer.tokenize(text))

# Run evaluation
for item in paragraph_data:
    src_lang, name, src, ref = item["lang"], item["name"], item["src"], item["ref"]
    print(f"\n=== {name} → English ===")
    ram_before, cpu_before = measure_system()
    pred, t_first, t_total, token_count = translate_m2m100(src, src_lang)
    ram_after, cpu_after = measure_system()

    # Accuracy metrics
    bleu_score   = bleu_mod.compute(predictions=[pred], references=[[ref]])["bleu"]
    bert_res     = bertscore_mod.compute(predictions=[pred], references=[ref], lang="en")
    meteor_score = meteor_mod.compute(predictions=[pred], references=[[ref]])["meteor"]
    rouge_score  = rouge_mod.compute(predictions=[pred], references=[ref])["rougeL"]
    ter_score    = ter_mod.compute(predictions=[pred], references=[[ref]])["score"]
    comet_res    = comet_mod.compute(sources=[src], predictions=[pred], references=[ref])
    comet_score  = next(v for v in comet_res.values() if isinstance(v, (int, float)))

    # Output
    print(f"Source    : {src}")
    print(f"Reference : {ref}")
    print(f"Predicted : {pred}\n")
    print(f"BLEU      : {bleu_score:.4f}")
    print(f"BERTScore : {bertscore_mod.compute(predictions=[pred], references=[ref], lang='en')['f1'][0]:.4f}")
    print(f"METEOR    : {meteor_score:.4f}")
    print(f"ROUGE-L   : {rouge_score:.4f}")
    print(f"TER       : {ter_score:.4f}")
    print(f"COMET     : {comet_score:.4f}\n")
    print(f"Time to 1st tok : {t_first:.4f}s")
    print(f"Total inf time  : {t_total:.4f}s")
    print(f"Tokens/sec      : {token_count / t_total:.2f}")
    if device.type == "cuda":
        print(f"VRAM used       : {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
    print(f"RAM   : {ram_after - ram_before:.2f} MB")
    print(f"CPU   : {cpu_after - cpu_before:.2f} %")
    print(f"Model size : {sum(p.numel() for p in model.parameters()) * 4 / 1024 ** 2:.2f} MB")
    print("=" * 60)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Source    : Jeg ansøger om et turistvisum for at besøge USA. Jeg har til hensigt at blive i to uger og besøge kendte vartegn som Frihedsgudinden og Times Square. Jeg vender tilbage til mit hjemland efter ferien.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I apply for a tourist visa to visit the United States. I intend to stay for two weeks and visit well-known signs such as the Goddess of Freedom and Times Square. I return to my homeland after the holiday.

BLEU      : 0.5467
BERTScore : 0.9586
METEOR    : 0.7395
ROUGE-L   : 0.7160
TER       : 31.7073
COMET     : 0.8658

Time to 1st tok : 18.0179s
Total inf time  : 18.0245s
Tokens/sec      : 2.88
RAM   : 0.00 MB
CPU   : -14.30 %
Model size : 1845.95 MB

=== German → English ===


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source    : Ich beantrage ein Touristenvisum, um die Vereinigten Staaten zu besuchen. Ich beabsichtige, zwei Wochen zu bleiben und bekannte Sehenswürdigkeiten wie die Freiheitsstatue und den Times Square zu besuchen. Nach meinem Urlaub kehre ich in mein Heimatland zurück.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I apply for a tourist visa to visit the United States. I intend to stay for two weeks and visit well-known attractions such as the Statue of Freedom and the Times Square. After my holiday I return to my homeland.

BLEU      : 0.5284
BERTScore : 0.9730
METEOR    : 0.7718
ROUGE-L   : 0.7073
TER       : 34.1463
COMET     : 0.8965

Time to 1st tok : 20.1286s
Total inf time  : 20.1352s
Tokens/sec      : 3.33
RAM   : 0.00 MB
CPU   : 14.00 %
Model size : 1845.95 MB

=== Estonian → English

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source    : Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I am planning to stay for two weeks and visit famous attractions such as Freedom House and Times Square. After a vacation I will return to my home country.

BLEU      : 0.3149
BERTScore : 0.9499
METEOR    : 0.5350
ROUGE-L   : 0.5429
TER       : 58.5366
COMET     : 0.8652

Time to 1st tok : 13.2290s
Total inf time  : 13.2340s
Tokens/sec      : 4.53
RAM   : 0.15 MB
CPU   : 11.00 %
Model size : 1845.95 MB
